# get aggregated tax from gather.
- How much additional content found when adding new MAGs
- What species yield the most additional content

In [19]:
import pandas as pd
from pathlib import Path

In [20]:
tax_out_folder = Path("/group/ctbrowngrp2/amhorst/2025-pig-corespp/results/tax_metag/summary")

all_agg = []

In [21]:
for csv_file in tax_out_folder.glob("*.csv"):
    # Load CSV (you can use chunksize if files are huge)
    df = pd.read_csv(csv_file)
    agg = (
        df[(df["rank"] == "species") & (df["lineage"] != "unclassified")]
        .assign(weighted_expr=df["f_weighted_at_rank"] * df["total_weighted_hashes"] * 10000)
        .groupby("lineage")
        .agg(avg_fraction=("weighted_expr", "mean"), n_found=("lineage", "size"))
        .reset_index()
    )

    # Append result
    all_agg.append(agg)

In [22]:
result = pd.concat(all_agg, ignore_index=True)
result.head()

,lineage,avg_fraction,n_found
0,d__Archaea;p__Asgardarchaeota;c__Njordarchaeia...,10000.0,1
1,d__Archaea;p__Methanobacteriota;c__Methanobact...,60000.0,1
2,d__Archaea;p__Methanobacteriota;c__Methanobact...,20000.0,1
3,d__Archaea;p__Methanobacteriota;c__Methanobact...,60000.0,1
4,d__Archaea;p__Methanobacteriota;c__Methanobact...,10000.0,1


In [31]:
# group accross all the tax files
df_final = (
    result.groupby("lineage")
    .agg(avg_fraction=("avg_fraction", "mean"), n_found=("n_found", "sum"))
    .reset_index()
)


In [34]:
df_final['n_found'].value_counts()

n_found
1      10390
2       4291
3       2608
4       2118
5       1352
       ...  
220        2
374        2
313        2
389        1
386        1
Name: count, Length: 390, dtype: int64